In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.1/752.1 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [1]:
import os
import shutil
import random
!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/MF_YOLOv8/

/content/drive/MyDrive/MF_YOLOv8


In [5]:
ls

data/         output/  SA-MF_YOLOv8.ipynb  training_results/  yolov8n.pt
dataset.yaml  runs/    test_images/        yolo_data/         yolov8s.pt


In [6]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [7]:
ls

data/         output/  SA-MF_YOLOv8.ipynb  training_results/  yolov8n.pt
dataset.yaml  runs/    test_images/        yolo_data/         yolov8s.pt


In [8]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/MF_YOLOv8/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 897 images---


  0%|          | 0/718 [00:00<?, ?it/s]

------ Training data created with 80% split 718 images -------


  0%|          | 0/179 [00:00<?, ?it/s]

------ Testing data created with a total of 179 images ----------
------ TASK COMPLETED -------


In [9]:
 ls

data/         output/  SA-MF_YOLOv8.ipynb  training_results/  yolov8n.pt
dataset.yaml  runs/    test_images/        yolo_data/         yolov8s.pt


In [10]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.4 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.0/78.2 GB disk)


In [11]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/MF_YOLOv8/dataset.yaml epochs=200 imgsz=640 batch=8 project=/content/drive/MyDrive/MF_YOLOv8/training_results name=memories

Ultralytics YOLOv8.2.4 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/MF_YOLOv8/dataset.yaml, epochs=200, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/MF_YOLOv8/training_results, name=memories4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_c

In [3]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/MF_YOLOv8/training_results/memories4/weights/best.pt conf=0.60 source=/content/drive/MyDrive/MF_YOLOv8/test_images name=predict8

Ultralytics YOLOv8.2.4 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/110 /content/drive/MyDrive/MF_YOLOv8/test_images/20160211_195344.jpg: 640x480 (no detections), 71.0ms
image 2/110 /content/drive/MyDrive/MF_YOLOv8/test_images/20170323_225222.jpg: 640x480 3 females, 12.2ms
image 3/110 /content/drive/MyDrive/MF_YOLOv8/test_images/20171217_200314.jpg: 480x640 2 females, 69.8ms
image 4/110 /content/drive/MyDrive/MF_YOLOv8/test_images/20180506_155550.jpg: 384x640 2 females, 67.9ms
image 5/110 /content/drive/MyDrive/MF_YOLOv8/test_images/20180506_183724.jpg: 384x640 1 female, 10.7ms
image 6/110 /content/drive/MyDrive/MF_YOLOv8/test_images/20180507_200329.jpg: 384x640 (no detections), 10.7ms
image 7/110 /content/drive/MyDrive/MF_YOLOv8/test_images/20180508_104807.jpg: 640x384 (no detections), 72.4ms
image 8/110 /content/drive/MyDrive/MF_YOLOv8/test_images/20180926_123120.jpg: 384x640 (

In [4]:
!cp -r runs/detect/predict8 /content/drive/MyDrive/MF_YOLOv8/output